In [1]:
import pickle
import torch
from torch import nn
import torchvision
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm
import csv
from collections import Counter
import numpy as np

In [2]:
from modules.utils import *

In [5]:
ensembl_trx = pickle.load(open('data/ensembl_trx.pkl', 'rb'))
#trx_orfs = pickle.load(open('data/trx_orfs.pkl', 'rb'))
#dataset = pickle.load(open('data/dataset.pkl', 'rb'))
#alt_dataset = pickle.load(open('data/alt_dataset.pkl', 'rb'))

In [7]:
alt_data = [(x,y['sequence']) for x,y in ensembl_trx.items() if x in alt_dataset]

In [8]:
def build_fasta(data, filename):
    fasta_sequences = []
    for i, trx in enumerate(data):
        trx_id, seq = trx[0], trx[1]
        header = f'>{trx_id}'
        fasta_sequence = header + '\n' + seq.upper()
        fasta_sequences.append(fasta_sequence)
    
    fasta_text = '\n'.join(fasta_sequences)
    
    with open(filename + '.fa', 'w') as fasta_file:
        fasta_file.write(fasta_text)

In [9]:
build_fasta(alt_data, f'mydb_alt_data')

In [27]:
trx_chr = dict()
for trx, orfs in trx_orfs.items():
    if trx not in dataset:
        continue
    chr_ = list(set([x['chromosome'] for x in orfs.values()]))[0]
    trx_chr[trx] = chr_

chr_set1 = ['1', '7', '13', '19']
chr_set2 = ['2', '8', '14', '20']
chr_set3 = ['3', '9', '15', '21']
chr_set4 = ['4', '10', '16', '22']
chr_set5 = ['5', '11', '17', 'X']
chr_set6 = ['6', '12', '18', 'Y']
for idx, chr_set in enumerate([chr_set1, chr_set2, chr_set3, chr_set4, chr_set5, chr_set6]):
    data = [(x,y['sequence']) for x,y in ensembl_trx.items() if x in dataset and trx_chr[x] in chr_set]
    build_fasta(data, f'mydb{idx}')

In [10]:
bins = pickle.load(open('data/bins_.pkl', 'rb'))

In [11]:
results = dict()
with open('results.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for n, row in enumerate(reader):
        trx = row[0]
        match = row[1]
        if trx == match:
            continue
        if trx not in results:
            results[trx] = [match]
        else:
            results[trx].append(match)

In [12]:
from tqdm.notebook import tqdm

In [13]:
alt_split = dict()
for trx, matches in tqdm(results.items()):
    alt_split[trx] = []
    for match in matches:
        for idx, bin_ in enumerate(bins):
            if match in bin_:
                alt_split[trx].append(idx)
alt_split = {x:Counter(y).most_common(1)[0][0] for x,y in alt_split.items()}

  0%|          | 0/7411 [00:00<?, ?it/s]

In [24]:
pickle.dump(alt_split, open('data/alt_split_idx.pkl', 'wb'))

In [14]:
for split in set([x for x in alt_split.values()]):
    X_test = [y['mapped_seq'] for x,y in alt_dataset.items() if x in alt_split and alt_split[x] == split]
    y_test = [y['mapped_cds'] for x,y in alt_dataset.items() if x in alt_split and alt_split[x] == split]
    test_split = (X_test,y_test)
    pickle.dump(test_split, open(f'data/alt_split{split}.pkl', 'wb'))

In [15]:
X_test = [y['mapped_seq'] for x,y in alt_dataset.items() if x not in alt_split]
y_test = [y['mapped_cds'] for x,y in alt_dataset.items() if x not in alt_split]
test_split = (X_test,y_test)
pickle.dump(test_split, open(f'data/alt_split10.pkl', 'wb'))